In [6]:
import torch
from parler_tts import ParlerTTSForConditionalGeneration
from transformers import AutoTokenizer
import soundfile as sf


In [7]:
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
device = "cpu"
model_path = "parler-tts-mini-v1"
model = ParlerTTSForConditionalGeneration.from_pretrained(
    "parler-tts-mini-v1"
    ).to(device)
tokenizer = AutoTokenizer.from_pretrained("parler-tts-mini-v1")

/opt/anaconda3/envs/MPS-Torch/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


In [13]:
prompt = "Hey, how are you doing today?"
description = "Laura's voice is monotone, with a very close recording that almost has no background noise."

input_ids = tokenizer(description, return_tensors="pt").input_ids.to(device)
prompt_input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

input_data = tokenizer(description, return_tensors="pt", padding=True, truncation=True)
prompt_data = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
# Use attention masks generated by the tokenizer
input_attention_mask = input_data.attention_mask.to(device)
prompt_attention_mask = prompt_data.attention_mask.to(device)

generation = model.generate(input_ids=input_ids, 
                            prompt_input_ids=prompt_input_ids,
                            attention_mask=input_attention_mask,    # Add attention masks here
                            # prompt_attention_mask=prompt_attention_mask # If supported by the model
                    )
audio_arr = generation.cpu().numpy().squeeze()
sf.write("parler_tts_out.wav", audio_arr, model.config.sampling_rate)